# Report 1


### library importation, constant definition and data loading from files

The next cell defines the imports nedeed to execute the notebook

In [ ]:
import pandas as pd
#!pip install geopandas
import geopandas as gpd
import matplotlib.pyplot as plt
%pylab inline
from math import sin, cos, sqrt, atan2, radians
pd.set_option('precision', 5)
from functools import reduce
import os
from operator import truediv
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

import matplotlib.pyplot as plt
import numpy as np

import os
import sys
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pareto

pd.set_option('display.float_format', lambda x: '%.1f' % x)

from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))


The next cell defines the contanst used by the procedures bellow


In [ ]:
report1_dir = '../dras-controller/output/'
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    display(x)
    #print(x.to_latex(index=False)) 
    pd.reset_option('display.max_rows')
    
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

def highlight_min(s):
    is_min = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_min]
    


In [ ]:
def get_dist(x_1,x_2,y_1,y_2):
  d = math.sqrt((x_1-x_2)*(x_1-x_2) + (y_1-y_2)*(y_1-y_2))
  return d

def get_dist_norm(x_1,x_2,y_1,y_2):
  d = math.sqrt((x_1-x_2)*(x_1-x_2) + (y_1-y_2)*(y_1-y_2))
  return d/((y_1-y_2) + (x_2-x_1))

def get_dist_norm2(x_1,x_2,y_1,y_2):
  #d = math.sqrt((x_1-x_2)/(x_2/y_2)*(x_1-x_2)/(x_2/y_2) + (y_1-y_2)*(y_1-y_2))
  d = math.sqrt((x_1-x_2)/3*(x_1-x_2)/3 + (y_1-y_2)*(y_1-y_2))
  #d= y_1 - y_2
  #d= x_2 - x_1
  return d


def changeLow(name):
    if "highest-deadline-first" in name:
        return "HDF"
    if "highest-penalty-first" in name:
        return "HPF"
    if "highest-mips-first" in name:
        return "LLF"
    if "lowest-id-first" in name:
        return "FCFS"


def getTipo(name):
    if "medium" in name:
        return "hom-hom"
    if "small" in name:
        return "hom-hom"
    if "large" in name:
        return "hom-hom"
    
    if "5-0-0" in name and "het" in name:
        return "hom-het"
    if "0-10-0" in name and "het" in name:
        return "hom-het"
    if "0-0-30" in name and "het" in name:
        return "hom-het"
    
    if "-20-" in name and "hom" in name:
        return "het-hom"
    if "-3-" in name and "hom" in name:
        return "het-hom"
    if "-6-" in name and "hom" in name:
        return "het-hom"
    
    if "-20-" in name and "het" in name:
        return "het-het"
    if "-3-" in name and "het" in name:
        return "het-het"
    if "-6-" in name and "het" in name:
        return "het-het"
    
    return name

def getSize(name):
    if "medium" in name:
        return "medium"
    if "small" in name:
        return "small"
    if "large" in name:
        return "large"
    
    if "5-0-0" in name and "het" in name:
        return "small"
    if "0-10-0" in name and "het" in name:
        return "medium"
    if "0-0-30" in name and "het" in name:
        return "large"
    
    if "-20-" in name and "hom" in name:
        return "large"
    if "-3-" in name and "hom" in name:
        return "small"
    if "-6-" in name and "hom" in name:
        return "medium"
    
    if "-20-" in name and "het" in name:
        return "large"
    if "-3-" in name and "het" in name:
        return "small"
    if "-6-" in name and "het" in name:
        return "medium"
    
    return name

def getHeuLow(name):
    return name.split("_")[1]

def getHeu(name):
    if "BTO-g" in name:
        return "BTO-g"
    if "BTO-d" in name:
        return "BTO-d"
    if "ideal" in name:
        return "ideal"
    if "nash" in name:
        return "NE"
    if "PMP" in name:
        return "PMP"
    if "bau" in name:
        return "BaU"
    return name

def getInstance(name):
    if "medium-tol1" in name:
        return name[-15:-4]
    if "medium" in name:
        return name[-17:-4]
    if "small-tol1" in name:
        return name[-14:-4]
    if "small" in name:
        return name[-16:-4]
    if "large-tol1" in name:
        return name[-14:-4]
    if "large" in name:
        return name[-16:-4]
    if "DRD" in name:
        return "DRD" + name.split("DRD")[1][:-4]



def fotmatTable(table):
    newTable = pd.DataFrame()
    newTable['heu'] = table['heu'] + '-'+ table.apply(lambda e:changeLow(e['low']),axis = 1)
    
    newTable['reduction'] = table['AVG clients reduction (W)'].round(1)
    newTable['cost'] = table['AVG dcCost'].round(1)
    newTable['#V'] = table['AVG #V'].round(1)
    newTable['VT'] = table['AVG VT'].round(1)

    newTable['RDI'] = table['AVG RDI'].round(3)
    newTable['dI'] = table['AVG dI'].round(1)
    newTable['#B'] = table['AVG #B'].round(1)
    newTable['s'] = table['AVG s'].round(1)

    newTable.style.apply(highlight_max, subset=['reduction','cost']).apply(highlight_min, subset=['VT'])
    
    return newTable


# plot client function

the scenarios are generated from swf files


In [ ]:

def printHeu(name):
    
    data = pd.DataFrame()
    for filename in [f for f in os.listdir(report1_dir +'/') if ".FUN" in f 
                     and not 'pareto' in f
                     and not 'ideal' in f
                    
                    ]:

        aux =  report1_dir + filename  
        dataAux = pd.read_csv(aux, header=None,sep=' ',skiprows=0,)
        
        dataAux['instance'] = getHeu(filename)
        dataAux['heu'] = getInstance(filename)
        dataAux['low'] = getHeuLow(filename)
        dataAux['tipo'] = getTipo(filename)
        dataAux['size'] = getSize(filename)



        #print(data)
        dataAux = dataAux.fillna(0)
        dataAux = dataAux.replace(NaN,'0.0')
        dataAux = dataAux.replace('y','0.0')
        dataAux = dataAux.replace('err','0.0')
        dataAux.columns = [
                'i','offer ($)', 'clients reduction (W)','motor','err','dcCost','socialCost','#V','VT',
                           'heu','instance','low','tipo','size']
        data = data.append(dataAux)
    
    data = data.reset_index().drop(columns=['index'])
    
    
    data = data[['clients reduction (W)','dcCost','#V','VT',
                           'heu','instance','low','tipo','size']]
    data = data.round(2)
    
    dataMean = data.copy()
    
    
    dataMean = dataMean.groupby(['heu','tipo','low','size']).mean()
    #dataMean = dataMean.groupby(['heu','tipo']).mean()

    dataMean = dataMean.reset_index()
    dataMean.columns = ['heu','tipo','low','size','AVG clients reduction (W)','AVG dcCost','AVG #V','AVG VT']
    #dataMean.columns = ['AVG heu','tipo','AVG clients reduction (W)','AVG dcCost','AVG VT','AVG VT']

    dataStd = data.copy()

    
    dataStd = dataStd.groupby(['heu','tipo','low','size']).std()
    #dataStd = dataStd.groupby(['heu','tipo']).std()

    dataStd = dataStd.reset_index()
    dataStd = dataStd[['clients reduction (W)','dcCost','#V','VT']]
    dataStd.columns = ['STD reduction (W)','STD dcCost','STD #V','STD VT']
    
    dataCount = data.copy()
    dataCount['count'] = 1
    dataCount = dataCount.groupby(['heu','tipo','low','size']).sum()
    #dataCount = dataCount.groupby(['heu','tipo']).sum()

    dataCount = dataCount.reset_index()
    dataCount = dataCount[['count']]

    
    data = pd.concat([dataMean, dataStd], axis=1)
    data = pd.concat([data, dataCount], axis=1)

    
    data = data.sort_values(['heu','tipo']) 
    #data = data.sort_values(['tipo'])  

    
    lows = ['highest-penalty-first', 'highest-mips-first','highest-deadline-first']
    data = data[data['low'].isin(lows)]

    #print_full(fotmatTable(data[data['size']==name]))
    print_full(data[data['size']==name])
           

        
print('small')
printHeu('small')

print('medium')
printHeu('medium')

print('large')
printHeu('large')







In [ ]:

def getIdealReduction(low,size,instance,data):
    return data[(data['instance']==instance) & (data['low']==low)
& (data['size']==size) & (data['heu']=='ideal')]['reduction']
    
def getIdealReductionGlobal(low,size,instance,data):
    aux = data[(data['instance']==instance)& (data['heu']=='ideal')& 
               (data['size']==size)]['reduction']
    aux = aux.sort_values(ascending=False)
    return aux.values[0]

def getNumberOfBestSolution(heu,low,dI,instance,data):
    aux = data.copy()
    aux = aux.groupby(['instance']).min()
    aux = aux.reset_index()
    
    aux = aux.apply(lambda x: 1 if dI == x.dI else 0,axis = 1)
    #aux = aux.apply(lambda x: 1 if dI == x.dI and not (x.heu==heu and x.low == low)  else 0,axis = 1)

    return aux.sum(axis = 0, skipna = True)

def getStrenght(reduction,cost,instance,data):
    aux = data.copy()
    ##aux = aux.groupby(['instance']).max()
    ##aux = aux.reset_index()
    aux3 = aux.apply(lambda x: 
                                  1 
                                  if instance == x.instance
                                  and reduction >= x.reduction
                                  and cost <= x.cost 
                                  else 0,axis = 1)
    # less myself
    suma = aux3.sum(axis = 0, skipna = True)
    return (suma - 1)




def printHeu(name):
    
    data = pd.DataFrame()
    for filename in [f for f in os.listdir(report1_dir +'/') if ".FUN" in f 
                     and not 'pareto' in f]:

        aux =  report1_dir + filename  
        dataAux = pd.read_csv(aux, header=None,sep=' ',skiprows=0,)
        
        dataAux['heu'] = getHeu(filename)
        dataAux['instance'] = getInstance(filename)
        dataAux['low'] = getHeuLow(filename)
        dataAux['tipo'] = getTipo(filename)
        dataAux['size'] = getSize(filename)



        #print(data)
        dataAux = dataAux.fillna(0)
        dataAux = dataAux.replace(NaN,'0.0')
        dataAux = dataAux.replace('y','0.0')
        dataAux = dataAux.replace('err','0.0')
        dataAux.columns = ['i','offer ($)', 'reduction','motor','err','cost','socialCost','#V','VT',
                           'heu','instance','low','tipo','size']
        data = data.append(dataAux)
    
    data = data.reset_index().drop(columns=['index'])
    
    
    data = data[['reduction','cost','#V','VT',
                           'heu','instance','low','tipo','size']]
    
    data = data.round(2)
    
    data = data[data['size']==name]
    data = data.reset_index().drop(columns=['index'])
    
    data['reduction_ideal'] = data.apply(lambda e:
                                         getIdealReduction(e['low'],e['size'],e['instance'],data).values[0]
                                         ,axis = 1)
    data['reduction_ideal_g'] = data.apply(lambda e:
                                         getIdealReductionGlobal(e['low'],e['size'],e['instance'],data)
                                         ,axis = 1)                                                 
              
    # delete ideal for normalizing
    data = data[data['heu']!='ideal']
    data = data.reset_index().drop(columns=['index'])
    
        
    data['RDI'] = data.apply(lambda e: get_dist_norm(0.0,e['cost'],
                                                    e.reduction_ideal,e['reduction']),axis = 1)
    
    
    
    # FILTER
    # only this heu
    lows = ['highest-penalty-first', 'highest-mips-first','highest-deadline-first','lowest-id-first']
    data = data[data['low'].isin(lows)]
    data = data.reset_index().drop(columns=['index'])
    
    # only one bau
    bau = ['BaU']
    lows = ['highest-penalty-first', 'highest-mips-first','highest-deadline-first']
    data = data[  ~(  (data['heu'].isin(bau)) & (data['low'].isin(lows)) )]
    data = data[  (  (data['heu'].isin(bau)) | (data['low'].isin(lows)) )]
    
    data = data.reset_index().drop(columns=['index'])
    
    print_full(data)
    data['dI'] = data.apply(lambda e: get_dist_norm2(0.0,e['cost'],
                                                     e.reduction_ideal_g,e['reduction']),axis = 1)
    
    
    
    # drop unnecesay columns
    data = data.drop(columns=['reduction_ideal'])
    data = data.drop(columns=['reduction_ideal_g'])

    
    data['#B'] = data.apply(lambda e: getNumberOfBestSolution(e.heu,e.low,e.dI,e.instance,data),axis = 1)
    data['s'] = data.apply(lambda e: getStrenght(e.reduction,e.cost,e.instance,data),axis = 1)
    
    #print_full(data.query("instance == 'DRD-3-1-1-het-1'"))
    
    
    dataMean = data.copy()
    dataMean = dataMean.groupby(['heu','low','size']).mean()
    #dataMean = dataMean.groupby(['heu','tipo']).mean()

    dataMean = dataMean.reset_index()
    dataMean.columns = ['heu','low','size','AVG clients reduction (W)',
                        'AVG dcCost','AVG #V','AVG VT','AVG RDI','AVG dI','AVG #B','AVG s']

    dataStd = data.copy()

    
    dataStd = dataStd.groupby(['heu','low','size']).std()
    #dataStd = dataStd.groupby(['heu','tipo']).std()

    dataStd = dataStd.reset_index()
    dataStd = dataStd[['reduction','cost','#V','VT','RDI','dI','#B','s']]
    dataStd.columns = ['STD reduction (W)','STD dcCost','STD #V','STD VT','STD RDI','STD dI','STD #B','STD s']
    
    dataCount = data.copy()
    dataCount['count'] = 1
    dataCount = dataCount.groupby(['heu','low','size']).sum()
    #dataCount = dataCount.groupby(['heu','tipo']).sum()

    dataCount = dataCount.reset_index()
    dataCount = dataCount[['count']]

    
    data = pd.concat([dataMean, dataStd], axis=1)
    data = pd.concat([data, dataCount], axis=1)

    
    data = data.sort_values(['heu','low']) 
    #data = data.sort_values(['tipo'])  
    
    
    data = fotmatTable(data[data['size']==name])


    
    #data['bau'] = data.apply(lambda e: get_dist(0.0,e['cost'],
   #                                                  e.reduction_ideal_g,e['reduction']),axis = 1)
    
    return data
               


#def getStrenght(data):
#    return data[(data['heu']==instance)
                
small = printHeu('small')
medium = printHeu('medium')
large = printHeu('large')







In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

small

In [9]:
def addMetrics(data,tasks):
    data['deltaNR'] = data.apply(lambda e: 
                                 e.reduction/data[data['heu']=='NE-HPF']['reduction']

                                     if (e.reduction > data[data['heu']=='NE-HPF']['reduction'].values[0])
                                     else (-1 * data[data['heu']=='NE-HPF']['reduction']/e.reduction)

                                     ,axis = 1)
    data['deltaNC'] = data.apply(lambda e: 
                                  -1 *  e.cost/data[data['heu']=='NE-HPF']['cost']   
                                     if (e.cost > data[data['heu']=='NE-HPF']['cost'].values[0])
                                     else (data[data['heu']=='NE-HPF']['cost']/e.cost)

                                     ,axis = 1)
    
    data['VT'] = data['VT'] / 3600
    data['#V'] = data['#V'] /  tasks* 100
    
    data['spider_V'] = 10 - data['#V'] / data['#V'].max() * 10 
    data['spider_reduction'] = data['reduction'] / data['reduction'].max() * 10 
    data['spider_dI'] = 10 - data['dI'] / data['dI'].max() * 10 
    data['spider_B'] = data['#B'] / data['#B'].max() * 10 
    
    data['deltaNR'] = data.apply(lambda e: 0 if e['deltaNR']<0 
                                        else e['deltaNR'] ,axis = 1)
    data['deltaNC'] = data.apply(lambda e: 0 if e['deltaNC']<0 
                                        else e['deltaNC'] ,axis = 1)
    
    data['spider_deltaNR'] = data['deltaNR'] / data['deltaNR'].max() * 10 
    data['spider_deltaNC'] = data['deltaNC'] / data[data['heu']!='BaU-FCFS']['deltaNC'].max() * 10

    

    
    
    #1, 3, 5, 7 y podría ser 9.1 (costo) y 9.2  3649.30/7669.90*10
    return data[['heu', '#V','reduction','dI','#B','deltaNR','deltaNC',
                 'spider_V', 'spider_reduction','spider_dI','spider_B','spider_deltaNR','spider_deltaNC']]
    
    return data

        
print_full(addMetrics(small.copy(),17375))
print_full(addMetrics(medium.copy(),57500))
print_full(addMetrics(large.copy(),218000))


small

,heu,#V,reduction,dI,#B,deltaNR,deltaNC,spider_V,spider_reduction,spider_dI,spider_B,spider_deltaNR,spider_deltaNC
0,BTO-d-HDF,4.239,909.200,3648.600,0.000,0.000,72.252,9.155,2.331,1.995,0.000,0.000,10.000
1,BTO-d-LLF,4.246,917.700,3640.300,0.000,0.000,51.883,9.153,2.353,2.013,0.000,0.000,7.181
2,BTO-d-HPF,50.144,3900.800,2683.900,0.500,1.810,0.000,0.000,10.000,4.111,10.000,10.000,0.000
3,BTO-g-HDF,4.237,907.200,3650.700,0.000,0.000,56.122,9.155,2.326,1.990,0.000,0.000,7.768
4,BTO-g-LLF,4.237,910.000,3647.900,0.000,0.000,57.922,9.155,2.333,1.996,0.000,0.000,8.017
5,BTO-g-HPF,43.438,3514.100,2737.500,0.200,1.631,0.000,1.337,9.009,3.994,4.000,9.009,0.000
6,BaU-FCFS,4.229,0.000,4557.800,0.000,0.000,inf,9.157,0.000,0.000,0.000,0.000,inf
7,NE-HDF,4.285,927.300,4120.800,0.000,0.000,0.000,9.145,2.377,0.959,0.000,0.000,0.000
8,NE-LLF,4.267,929.300,4118.900,0.000,0.000,0.000,9.149,2.382,0.963,0.000,0.000,0.000
9,NE-HPF,21.631,2154.800,2955.600,0.200,0.000,1.000,5.686,5.524,3.515,4.000,0.000,0.138


,heu,#V,reduction,dI,#B,deltaNR,deltaNC,spider_V,spider_reduction,spider_dI,spider_B,spider_deltaNR,spider_deltaNC
0,BTO-d-HDF,4.091,2169.300,11731.600,0.000,0.000,67.129,9.273,1.661,1.560,0.000,0.000,8.924
1,BTO-d-LLF,4.088,2189.600,11711.300,0.000,0.000,75.224,9.274,1.676,1.575,0.000,0.000,10.000
2,BTO-d-HPF,50.581,11968.500,8714.000,0.400,1.970,0.000,1.018,9.163,3.731,10.000,9.163,0.000
3,BTO-g-HDF,4.081,2142.200,11758.600,0.000,0.000,70.731,9.275,1.640,1.541,0.000,0.000,9.403
4,BTO-g-LLF,4.083,2178.800,11722.800,0.000,0.000,57.634,9.275,1.668,1.567,0.000,0.000,7.662
5,BTO-g-HPF,42.733,10429.800,8549.000,0.300,1.717,0.000,2.411,7.985,3.850,7.500,7.985,0.000
6,BaU-FCFS,4.076,0.000,13900.600,0.000,0.000,inf,9.276,0.000,0.000,0.000,0.000,inf
7,NE-HDF,4.121,2211.700,13060.200,0.000,0.000,0.000,9.268,1.693,0.605,0.000,0.000,0.000
8,NE-LLF,4.122,2234.000,13040.000,0.000,0.000,0.000,9.268,1.710,0.619,0.000,0.000,0.000
9,NE-HPF,20.898,6074.000,9339.800,0.200,0.000,1.000,6.289,4.650,3.281,5.000,0.000,0.133


,heu,#V,reduction,dI,#B,deltaNR,deltaNC,spider_V,spider_reduction,spider_dI,spider_B,spider_deltaNR,spider_deltaNC
0,BTO-d-HDF,4.157,7365.200,43995.900,0.000,0.000,66.449,9.298,1.461,1.434,0.000,0.000,9.566
1,BTO-d-LLF,4.156,7447.200,43913.900,0.000,0.000,69.468,9.298,1.478,1.450,0.000,0.000,10.000
2,BTO-d-HPF,51.771,44986.600,32800.500,0.500,2.353,0.000,1.255,8.926,3.614,10.000,8.926,0.000
3,BTO-g-HDF,4.150,7258.500,44102.800,0.000,0.000,63.620,9.299,1.440,1.413,0.000,0.000,9.158
4,BTO-g-LLF,4.150,7371.700,43989.600,0.000,0.000,65.909,9.299,1.463,1.435,0.000,0.000,9.488
5,BTO-g-HPF,45.254,40138.700,32969.200,0.400,2.099,0.000,2.355,7.964,3.581,8.000,7.964,0.000
6,BaU-FCFS,4.147,0.000,51360.500,0.000,0.000,inf,9.299,0.000,0.000,0.000,0.000,inf
7,NE-HDF,4.183,7509.500,47142.400,0.000,0.000,0.000,9.293,1.490,0.821,0.000,0.000,0.000
8,NE-LLF,4.184,7604.200,47054.300,0.000,0.000,0.000,9.293,1.509,0.838,0.000,0.000,0.000
9,NE-HPF,17.467,19120.300,35763.600,0.100,0.000,1.000,7.049,3.794,3.037,2.000,0.000,0.144


,heu,reduction,cost,#V,VT,RDI,dI,#B,s
0,BTO-d-HDF,909.200,67.500,736.600,244.900,0.739,3648.600,0.000,1.500
1,BTO-d-LLF,917.700,94.000,737.700,520.600,0.787,3640.300,0.000,2.600
2,BTO-d-HPF,3900.800,7616.100,8712.500,944576.200,0.900,2683.900,0.500,2.200
3,BTO-g-HDF,907.200,86.900,736.100,202.900,0.753,3650.700,0.000,0.900
4,BTO-g-LLF,910.000,84.200,736.200,251.800,0.771,3647.900,0.000,0.900
5,BTO-g-HPF,3514.100,7337.700,7547.400,800885.000,0.871,2737.500,0.200,1.200
6,BaU-FCFS,0.000,0.000,734.800,0.000,1.000,4557.800,0.000,0.000
7,NE-HDF,927.300,5711.500,744.500,1496.900,0.995,4120.800,0.000,0.500
8,NE-LLF,929.300,5710.800,741.400,1133.000,0.996,4118.900,0.000,0.500
9,NE-HPF,2154.800,4877.000,3758.400,313435.300,0.775,2955.600,0.200,2.100


In [8]:
data

NameError: name 'data' is not defined